In [256]:
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import dill

In [257]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import word_tokenize, pos_tag, sent_tokenize, ne_chunk
from nltk.tag import hmm
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/xhapa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/xhapa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/xhapa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/xhapa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/xhapa/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/xhapa/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/xhapa/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [258]:
from conllu import parse_incr

# Tokenize

In [259]:
text_en = 'Elon Musk said the incident Wednesday when Senate Minority Leader Mitch McConnell, R-Ky., appeared to freeze up during a news conference with other Republican leaders should warrant a "need" for a constitutional amendment.Musk, who owns Twitter and other companies, tweeted early Thursday morning, "We need a constitutional amendment. This is insane."The comment came in response to another user who provided a video of the incident showing McConnell addressing a group of reporters before abruptly stopping, staring blankly into the group, and standing idle for a brief moment. A Republican colleague broke McConnell’s apparent trance and helped escort him away from the podium.Musk’s tweet did not provide specifics on what the potential amendment would say or do. Several people who responded suggested there should be age limits for members of Congress.'
text_tokens_en = word_tokenize(text_en)

In [260]:
text_es = 'El ADN desvela el árbol genealógico más antiguo de una familia, hace 6.700 años El estudio genético de casi 100 cadáveres en una necrópolis de Francia reconstruye siete generaciones de un clan del Neolítico en el que los hombres se quedaban toda la vida en su lugar de origen mientras las mujeres se iban a otros grupos Entre 2004 y 2007, un grupo de arqueólogos excavó en Francia un cementerio de hace 6.700 años donde aparecieron más de 100 cadáveres de todas las edades. Las tumbas eran individuales y se habían excavado con cuidado para que ninguna quedase encima de otra. Apenas había objetos de valor junto a los muertos, algo raro, pues en aquella época estaba triunfando el Neolítico, la revolución que trajo a Europa el sedentarismo, la agricultura y la desigualdad. Por primera vez en la historia se pudo acumular grandes cantidades de alimento, y las primeras riquezas. Por razones desconocidas, grupos cada vez más grandes comenzaron a reunirse para levantar espectaculares monumentos megalíticos y tumbas donde se enterraba a las élites junto a objetos valiosos o sagrados, como armas y animales. En cambio, el cementerio francés parecía el de la gente corriente del momento. Ahora, un grupo de científicos francoalemanes ha conseguido extraer ADN de 94 cadáveres del cementerio francés para obtener su genoma completo. Las secuencias genéticas han dibujado los lazos de parentesco entre los fallecidos hasta componer un árbol de familia que se remonta siete generaciones; el mayor y más antiguo que se conoce de nuestra especie. Los resultados se publican hoy en Nature, referente de la mejor ciencia mundial. Los investigadores han encontrado que en este cementerio de Gurgy Les Noisats, al sur de París, hay dos grandes clanes familiares, encabezados por dos hombres. El más grande comprende a 64 familiares y abarca siete generaciones, todos enterrados en el mismo lugar. El segundo grupo lo forman 12 familiares de cinco generaciones. Apenas se ha descubierto un cruce entre los dos grupos: una de las madres del clan pequeño era pariente de un hombre del grande. El ADN de este colectivo que vivió hace casi siete milenios abre una ventana única para entender la familia, la sociedad y la cultura de una época tan importante como desconocida, pues aún no había escritura. El trabajo muestra un fenómeno clarísimo: los hombres de la familia se quedaban en su lugar de nacimiento para toda la vida, mientras las mujeres dejaban el seno familiar para ir a vivir con otros grupos. Los isótopos de estroncio acumulados en los dientes indican de dónde proviene el agua que bebió una persona durante su niñez, y los de las mujeres enterradas en Gurgy son de muchos lugares distintos. En cambio, apenas hay mujeres que fueran parte de los dos clanes originales del lugar. Estos hallazgos refuerzan una tendencia observada en otros yacimientos neolíticos posteriores: los hombres se quedaban y las mujeres se marchaban a vivir y formar familias en otros grupos, una práctica común en humanos y otros primates conocida como patrilocalidad y que evita los problemas asociados a la endogamia. La paleogenetista Maïte Rivollat, primera autora del estudio, destaca otro hallazgo sorprendente: “Hemos encontrado parejas que tuvieron muchos hijos. En un caso vemos hasta seis hermanos que vivieron hasta la edad adulta, y a su vez tuvieron varios hijos, lo que supone una familia muy extensa. Probablemente, tuvieron también hermanas cuyos restos no están aquí, pues se fueron a vivir a otros grupos”. Para su equipo, esto indica una gran fertilidad de las mujeres e implica que había abundancia de alimentos y probablemente estabilidad social. De hecho, no hay ni un signo de violencia en ninguno de los más de 100 cadáveres del cementerio. Apenas se han encontrado medio hermanos en ninguno de los dos clanes familiares. Tampoco se observa que los viudos y las viudas se emparejasen con sus cuñados. Esto implica que las parejas eran monógamas y que ya había una idea clara de evitar tener hijos con parientes cercanos.'
text_tokens_es = word_tokenize(text_es)

# Tagger

## Spanish

In [261]:
def parse_data_file(file, tagtype):
    sent_list = []

    for token_list in parse_incr(file):
        word_list = []
        for token in token_list:
            word_list.append((token['form'].lower(), token[tagtype]))
        sent_list.append(word_list)
    
    return sent_list

In [262]:
data_file = open('./datasets/UD_Spanish-AnCora/es_ancora-ud-train.conllu', encoding='utf-8')
tagtype = 'upos'
data = parse_data_file(data_file, tagtype)
data[1][:5]

[('según', 'ADP'),
 ('el', 'DET'),
 ('informe', 'NOUN'),
 (',', 'PUNCT'),
 ('el', 'DET')]

In [263]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, test_size=0.2, shuffle=True)

In [264]:
tagger_es = hmm.HiddenMarkovModelTagger.train(train_set)
predicted_set = tagger_es.tag(text_tokens_es)
tagger_es.accuracy(test_set)


0.9343390850423671

## English

In [265]:
pos_tag(text_tokens_en)

[('Elon', 'NNP'),
 ('Musk', 'NNP'),
 ('said', 'VBD'),
 ('the', 'DT'),
 ('incident', 'NN'),
 ('Wednesday', 'NNP'),
 ('when', 'WRB'),
 ('Senate', 'NNP'),
 ('Minority', 'NNP'),
 ('Leader', 'NNP'),
 ('Mitch', 'NNP'),
 ('McConnell', 'NNP'),
 (',', ','),
 ('R-Ky.', 'NNP'),
 (',', ','),
 ('appeared', 'VBD'),
 ('to', 'TO'),
 ('freeze', 'VB'),
 ('up', 'RP'),
 ('during', 'IN'),
 ('a', 'DT'),
 ('news', 'NN'),
 ('conference', 'NN'),
 ('with', 'IN'),
 ('other', 'JJ'),
 ('Republican', 'JJ'),
 ('leaders', 'NNS'),
 ('should', 'MD'),
 ('warrant', 'VB'),
 ('a', 'DT'),
 ('``', '``'),
 ('need', 'NN'),
 ("''", "''"),
 ('for', 'IN'),
 ('a', 'DT'),
 ('constitutional', 'JJ'),
 ('amendment.Musk', 'NN'),
 (',', ','),
 ('who', 'WP'),
 ('owns', 'VBZ'),
 ('Twitter', 'NNP'),
 ('and', 'CC'),
 ('other', 'JJ'),
 ('companies', 'NNS'),
 (',', ','),
 ('tweeted', 'VBD'),
 ('early', 'JJ'),
 ('Thursday', 'NNP'),
 ('morning', 'NN'),
 (',', ','),
 ('``', '``'),
 ('We', 'PRP'),
 ('need', 'VBP'),
 ('a', 'DT'),
 ('constitutional

## Export tagger_es model

In [266]:
with open('hmm_tagger_es.dill', 'wb') as file:
    dill.dump(tagger_es, file)

In [267]:
with open('hmm_tagger_es.dill', 'rb') as file:
    loaded_tagger = dill.load(file)

loaded_tagger.tag(text_tokens_es)


[('El', 'PRON'),
 ('ADN', 'AUX'),
 ('desvela', 'VERB'),
 ('el', 'DET'),
 ('árbol', 'NOUN'),
 ('genealógico', 'PUNCT'),
 ('más', 'ADV'),
 ('antiguo', 'ADJ'),
 ('de', 'ADP'),
 ('una', 'DET'),
 ('familia', 'NOUN'),
 (',', 'PUNCT'),
 ('hace', 'VERB'),
 ('6.700', 'NUM'),
 ('años', 'NOUN'),
 ('El', 'ADJ'),
 ('estudio', 'NOUN'),
 ('genético', 'ADJ'),
 ('de', 'ADP'),
 ('casi', 'ADV'),
 ('100', 'NUM'),
 ('cadáveres', 'NOUN'),
 ('en', 'ADP'),
 ('una', 'DET'),
 ('necrópolis', 'NOUN'),
 ('de', 'ADP'),
 ('Francia', 'NOUN'),
 ('reconstruye', 'ADP'),
 ('siete', 'NUM'),
 ('generaciones', 'NOUN'),
 ('de', 'ADP'),
 ('un', 'DET'),
 ('clan', 'NOUN'),
 ('del', '_'),
 ('Neolítico', 'VERB'),
 ('en', 'ADP'),
 ('el', 'DET'),
 ('que', 'PRON'),
 ('los', 'DET'),
 ('hombres', 'NOUN'),
 ('se', 'PRON'),
 ('quedaban', 'VERB'),
 ('toda', 'DET'),
 ('la', 'DET'),
 ('vida', 'NOUN'),
 ('en', 'ADP'),
 ('su', 'DET'),
 ('lugar', 'NOUN'),
 ('de', 'ADP'),
 ('origen', 'NOUN'),
 ('mientras', 'CCONJ'),
 ('las', 'DET'),
 ('mujeres

# NLTK text pre-processing

In [268]:
class NLTKPreProcessing():
    def __init__(self, text, lang) -> None:
        self.text = text  
        self.lang = lang

    def remove_html_tags(self):
        return self
    
    def to_lower(self):
        self.text = self.text.lower()
        return self

    def remove_double_spaces(self):
        self.text = [words for words in self.text if re.sub(' +', ' ', words)]
        return self

    def tokenize(self):
        self.text = word_tokenize(self.text)
        return self
    
    def pos(self):
        if self.lang == 'es':
            self.text =  tagger_es.tag(self.text)
        elif self.lang == 'en':
            self.text =  pos_tag(self.text)
        
        return self

    def remove_stopwords(self):
        if self.lang == 'es':
            self.stopwd = stopwords.words('spanish')
        elif self.lang == 'en':
            self.stopwd = stopwords.words('english')
        
        self.text = [(word, tag) for word, tag in self.text if word not in self.stopwd]

        return self

    def get_preprocessed(self):
        return self.text


In [269]:
from sklearn.base import BaseEstimator, TransformerMixin
class NLTKTextPreprocessor(TransformerMixin, BaseEstimator):
  def __init__(self):
    pass

  def fit(self, text):
    return self

  def transform(self, text):
    txt_preproc = NLTKPreProcessing(text, 'en')
    processed_text = \
        txt_preproc \
        .remove_html_tags()\
        .to_lower()\
        .tokenize()\
        .remove_double_spaces()\
        .pos()\
        .remove_stopwords()\
        .get_preprocessed()

    return processed_text

In [270]:
from sklearn.pipeline import Pipeline
pure_transformation_pipeline = Pipeline(steps=[
           ('text_preproc', NLTKTextPreprocessor())])
tfidf_data = pure_transformation_pipeline.fit_transform(text_en)
tfidf_data

[('elon', 'NN'),
 ('musk', 'NN'),
 ('said', 'VBD'),
 ('incident', 'NN'),
 ('wednesday', 'NN'),
 ('senate', 'JJ'),
 ('minority', 'NN'),
 ('leader', 'NN'),
 ('mitch', 'NN'),
 ('mcconnell', 'NN'),
 (',', ','),
 ('r-ky.', 'JJ'),
 (',', ','),
 ('appeared', 'VBD'),
 ('freeze', 'VB'),
 ('news', 'NN'),
 ('conference', 'NN'),
 ('republican', 'JJ'),
 ('leaders', 'NNS'),
 ('warrant', 'VB'),
 ('``', '``'),
 ('need', 'NN'),
 ("''", "''"),
 ('constitutional', 'JJ'),
 ('amendment.musk', 'NN'),
 (',', ','),
 ('owns', 'VBZ'),
 ('twitter', 'NN'),
 ('companies', 'NNS'),
 (',', ','),
 ('tweeted', 'VBD'),
 ('early', 'JJ'),
 ('thursday', 'NN'),
 ('morning', 'NN'),
 (',', ','),
 ('``', '``'),
 ('need', 'VBP'),
 ('constitutional', 'JJ'),
 ('amendment', 'NN'),
 ('.', '.'),
 ('insane', 'JJ'),
 ('.', '.'),
 ('``', '``'),
 ('comment', 'NN'),
 ('came', 'VBD'),
 ('response', 'NN'),
 ('another', 'DT'),
 ('user', 'NN'),
 ('provided', 'VBD'),
 ('video', 'NN'),
 ('incident', 'NN'),
 ('showing', 'VBG'),
 ('mcconnell', '

# NLTK Lemmatization

In [271]:
from nltk.stem import WordNetLemmatizer
import spacy.cli
spacy.cli.download("es_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 33.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [272]:
class NLTKLemmatization():
    def __init__(self, text, lang) -> None:
        self.text = text
        self.lang = lang
        self.lemmatizer = WordNetLemmatizer()
        self.nlp = spacy.load("es_core_news_sm")

    def get_wordnet_pos(self, tag):
        tag = tag[0].upper()
        tag_dict = {
            'J': 'a',  # Adjective
            'V': 'v',  # Verb
            'N': 'n',  # Noun
            'R': 'r'   # Adverb
        }
        return tag_dict.get(tag, 'n')
    

    def get_wordnet_upos(self, upos_tag):
        tag_dict = {
            'NOUN': 'n',  # Noun
            'VERB': 'v',  # Verb
            'ADJ': 'a',   # Adjective
            'ADV': 'r'    # Adverb
        }
        return tag_dict.get(upos_tag, 'n')  

    def lemmatize(self):
        if self.lang == 'en':
            self.text = [self.lemmatizer.lemmatize(word, pos=self.get_wordnet_pos(tag)) for word, tag in self.text]
        elif self.lang == 'es':
            words_with_space = [word + ' ' for word, tag in self.text]
            doc = self.nlp(''.join(words_with_space))
            self.text = [token.lemma_ for token in doc]

        return self
    
    def get_lemmatization(self):
        return  self.text

In [273]:
class NLTKLemmatizationProcessor(TransformerMixin, BaseEstimator):
  def __init__(self):
    pass

  def fit(self, text):
    return self

  def transform(self, text):
    txt_lemma = NLTKLemmatization(text, 'en')
    processed_text = \
        txt_lemma \
        .lemmatize()\
        .get_lemmatization()

    return processed_text

In [274]:
lemmatization_pipeline = Pipeline(steps=[
           ('text_lemma', NLTKLemmatizationProcessor())])
data = lemmatization_pipeline.fit_transform(tfidf_data)
data

['elon',
 'musk',
 'say',
 'incident',
 'wednesday',
 'senate',
 'minority',
 'leader',
 'mitch',
 'mcconnell',
 ',',
 'r-ky.',
 ',',
 'appear',
 'freeze',
 'news',
 'conference',
 'republican',
 'leader',
 'warrant',
 '``',
 'need',
 "''",
 'constitutional',
 'amendment.musk',
 ',',
 'own',
 'twitter',
 'company',
 ',',
 'tweet',
 'early',
 'thursday',
 'morning',
 ',',
 '``',
 'need',
 'constitutional',
 'amendment',
 '.',
 'insane',
 '.',
 '``',
 'comment',
 'come',
 'response',
 'another',
 'user',
 'provide',
 'video',
 'incident',
 'show',
 'mcconnell',
 'address',
 'group',
 'reporter',
 'abruptly',
 'stop',
 ',',
 'star',
 'blankly',
 'group',
 ',',
 'stand',
 'idle',
 'brief',
 'moment',
 '.',
 'republican',
 'colleague',
 'break',
 'mcconnell',
 '’',
 'apparent',
 'trance',
 'help',
 'escort',
 'away',
 'podium.musk',
 '’',
 'tweet',
 'provide',
 'specific',
 'potential',
 'amendment',
 'would',
 'say',
 '.',
 'several',
 'people',
 'respond',
 'suggest',
 'age',
 'limit',
 '

# Keywords and Collocations

In [275]:
from nltk.probability import FreqDist
num_keywords = 10
threshold = 1
words = [word for word, tag in tfidf_data]
fdist = FreqDist(words)
keywords = [word for word, _ in fdist.most_common(num_keywords) if len(word) > threshold]
keywords

['mcconnell',
 '``',
 'incident',
 'republican',
 'need',
 'constitutional',
 'amendment',
 'group']

In [281]:
from nltk.collocations import *
bigrams_measure = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(text_tokens_en)
finder.nbest(bigrams_measure.pmi, 10)

[('Elon', 'Musk'),
 ('Leader', 'Mitch'),
 ('Minority', 'Leader'),
 ('Musk', 'said'),
 ('Senate', 'Minority'),
 ('Several', 'people'),
 ('The', 'comment'),
 ('This', 'is'),
 ('Thursday', 'morning'),
 ('Wednesday', 'when')]

In [284]:
stopwd = stopwords.words('english')
[(word1, word2) for word1, word2 in finder.nbest(bigrams_measure.pmi, 10) if ((word1 not in stopwd) and (word2 not in stopwd))]

[('Elon', 'Musk'),
 ('Leader', 'Mitch'),
 ('Minority', 'Leader'),
 ('Musk', 'said'),
 ('Senate', 'Minority'),
 ('Several', 'people'),
 ('The', 'comment'),
 ('Thursday', 'morning')]

# NER

In [278]:
from nltk.tree import Tree
class NLTKEntityPreProcessing(NLTKPreProcessing):
    def __init__(self, text, lang) -> None:
        super().__init__(text, lang)

    def get_continuous_chunks(self):
        chunked = ne_chunk(self.text)
        self.continuous_chunks = []
        current_chunk = []
        for i in chunked:
                if type(i) == Tree:
                        current_chunk.append(" ".join([token for token, pos in i.leaves()]))
                if current_chunk:
                        named_entity = " ".join(current_chunk)
                        if named_entity not in self.continuous_chunks:
                                self.continuous_chunks.append(named_entity)
                                current_chunk = []
                else:
                        continue
        return self

    def get_preprocessed(self):
        return self.continuous_chunks

In [279]:
from sklearn.base import BaseEstimator, TransformerMixin
class NLTKEntityPreprocessor(TransformerMixin, BaseEstimator):
  def __init__(self):
    pass

  def fit(self, text):
    return self

  def transform(self, text):
    txt_preproc = NLTKEntityPreProcessing(text, 'en')
    processed_text = \
        txt_preproc \
        .tokenize()\
        .pos()\
        .get_continuous_chunks()\
        .get_preprocessed()

    return processed_text

In [280]:
from sklearn.pipeline import Pipeline
entity_transformation_pipeline = Pipeline(steps=[
           ('text_preproc', NLTKEntityPreprocessor())])
ent_data = entity_transformation_pipeline.fit_transform(text_en)
ent_data

['Elon',
 'Musk',
 'Senate',
 'Mitch McConnell',
 'Republican',
 'Twitter',
 'McConnell',
 'Republican McConnell',
 'Congress']

# Sentiment model classification